In [2]:
import delta
import delta.pip_utils
import delta.tables
import pyspark
import pyarrow as pa
import pyspark.pandas as ps

def get_spark():
    builder = (
        pyspark.sql.SparkSession.builder.appName("lakehouse")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
    )
    return delta.pip_utils.configure_spark_with_delta_pip(builder).getOrCreate()
